# Time Series Classification Using Transformers

- From Kaggle competition:
    - [Land USe / Land Cover Time Series](https://www.kaggle.com/datasets/saraivaufc/land-use-land-cover-time-series);

## Initial Setup

In [1]:
%%time

import math
import torch
import torch.nn as nn

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

%matplotlib inline

CPU times: user 2.15 s, sys: 1.03 s, total: 3.18 s
Wall time: 1.62 s


## Explore Available Data

In [2]:
%%time

df = pd.read_csv("./time_series.csv")

# Show it.
df

CPU times: user 454 ms, sys: 101 ms, total: 555 ms
Wall time: 556 ms


,id,date,red,nir,swir,class
0,0,2017-06-10,0.125,0.263,0.249,5
1,0,2017-06-26,0.149,0.283,0.254,5
2,0,2017-07-12,0.170,0.304,0.284,17
3,0,2017-07-28,0.191,0.319,0.302,17
4,0,2017-08-13,0.213,0.343,0.319,17
...,...,...,...,...,...,...
1264995,139992,2020-07-27,0.160,0.305,0.263,2
1264996,139992,2020-08-12,0.189,0.354,0.298,17
1264997,139992,2020-08-28,0.216,0.376,0.305,17
1264998,139992,2020-09-13,0.231,0.389,0.332,17


In [3]:
# Random selection for time series.
id_random = np.random.choice(df.id.unique())
df_time_series_unique = df[df.id == id_random]
df_time_series_unique.set_index(keys=["date"], inplace=True)
df_time_series_unique.index = df_time_series_unique.index.astype("datetime64[ns]")

print(f">>> Time series id={id_random} has {df_time_series_unique.shape[0]} time stamps.")

# Show it.
df_time_series_unique

>>> Time series id=2110 has 23 time stamps.


,id,red,nir,swir,class
date,,,,,
2017-06-10,2110,0.185,0.250,0.345,17
2017-06-26,2110,0.189,0.249,0.350,17
2017-07-12,2110,0.198,0.257,0.372,17
2017-07-28,2110,0.201,0.259,0.382,17
2017-08-13,2110,0.215,0.272,0.404,17
2017-08-29,2110,0.226,0.280,0.407,17
2017-09-14,2110,0.231,0.287,0.402,17
2017-09-30,2110,0.230,0.286,0.396,17
2017-10-16,2110,0.245,0.314,0.384,17


## Model Definition

### Main Parameters 

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # CPU or GPU?

INPUT_WINDOW = 1  # Sequence lenght.
OUTPUT_WINDOW = 1 # Predicted steps.
BATCH_SIZE = 64   # Batch size.
N_FEATURES = 4    # Number of bands.

print(f">>> Device: {DEVICE}")

>>> Device: cpu


### Positional Enconder

In [5]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model=N_FEATURES, max_len=INPUT_WINDOW):
        
        super(PositionalEncoding, self).__init__()       
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pe", pe)

    def forward(self, x):
        
        return x + self.pe[:x.size(0), :]